In [1]:
import json
import time

from duckdb.duckdb import keyword
from tqdm.auto import tqdm
import requests
import pandas as pd
import geopandas as gpd
from docx import Document
from keybert import KeyBERT


tqdm.pandas()
client_cert = "20250129_IDU_ADGX_LeonTur.crt"
ca_cert = "onti-ca.crt"
client_key = "DECFILE"

In [19]:
def request_from_llm(prompt: str) -> str:
    """
    Function extracts prompt from llm api
    Args:
        prompt (str): prompt to execute
    Returns:
        str: request from llm
    """

    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "model": "llama3.3",
        "prompt": prompt,
        "stream": False,
        "temperature": 0.2
    }
    response = requests.post(
        url="http://a.dgx:11434/api/generate",
        headers=headers,
        data=json.dumps(data),
        cert=(client_cert, client_key),
        verify=ca_cert
    )
    if response.status_code == 200:
        return response.json()["response"]
    return ""

In [23]:
def construct_lemma_prompt(text: str) -> str:

    prompt = f"""
        Приведи слово "{text}" в первоначальную форму.
        Если слово не было передано в тексте верни ''
        """
    return prompt

def get_tags(text: str) -> str:

    prompt = construct_lemma_prompt(text)
    tags_from_llm = request_from_llm(prompt)
    return tags_from_llm

In [4]:
df = pd.read_csv("with_names.csv")
test_df = df[:100].copy()

In [6]:
test_df["tags"] = test_df["text"].progress_apply(get_tags)

  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
hole_text = "\n".join(test_df["text"].to_list())
kw_model = KeyBERT()

In [ ]:
def extract_keywords(text: str, kw_model: KeyBERT) -> list[str]:

    tags_num = 1 + round(len(text) / 500)
    tags_data = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), top_n=tags_num)
    tags = [i[0] for i in tags_data if i[1] > 0.5]
    cleaned_tags = [get_tags(tag) for tag in tags]
    return cleaned_tags

res = test_df["text"].progress_apply(extract_keywords, kw_model=kw_model)

  0%|          | 0/100 [00:00<?, ?it/s]

In [22]:
res

0     [Это задача по морфологическому анализу и норм...
1     [Я готов помочь!\n\nОднако, я не вижу слова "С...
2     [Слово "стратегии" в первоначальной форме - "с...
3     [Вы предоставили только одно слово: "социально...
4     [Прошу прощения, но я не вижу списка слов для ...
                            ...                        
95                                                   []
96                                                   []
97                                                   []
98                                                   []
99    [аэропорт\nПулково, 'аэропорт Пулково' вернетс...
Name: text, Length: 100, dtype: object

In [11]:
from keybert import KeyBERT

kw_model = KeyBERT()
test_df["text"].apply(kw_model.extract_keywords, keyphrase_ngram_range=(1, 2), stop_words=None, top_n=3)

0                                   [(приложение, 1.0)]
1     [(петербурга, 0.7613), (санкт, 0.7016), (закон...
2                                 [(стратегии, 0.9097)]
3       [(социально, 0.8148), (экономического, 0.7896)]
4     [(развития, 0.7858), (петербурга, 0.7103), (са...
                            ...                        
95    [(внешнеэкономических, 0.396), (российской, 0....
96    [(грузы, 0.3667), (железнодорожный, 0.3454), (...
97    [(грузовом, 0.3545), (грузы, 0.3413), (грузов,...
98    [(перевозки, 0.3178), (переместились, 0.3109),...
99    [(аэропорт, 0.4723), (аэропортом, 0.4666), (со...
Name: text, Length: 100, dtype: object